<a href="https://colab.research.google.com/github/Ori1234/JA-RNN/blob/master/Thesis_8_march.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://webcache.googleusercontent.com/search?q=cache:viNLSTwuTS0J:https://www.reddit.com/r/datascience/comments/bkrzah/google_colab_how_to_avoid_timeoutdisconnect_issues/+&cd=2&hl=en&ct=clnk&gl=il

Go to the google Colab console (ctrl+shift+i)

Dont exit the console until you get "Working" as the output in the console window.


Note to self: Make sure you dont run anything for more than 12 hrs on Colab






```

function ClickConnect(){console.log("Working");if (document.querySelector("paper-button#ok")!=null){document.querySelector("paper-button#ok").click()}}val=setInterval(ClickConnect,60000)

clearInterval(val)
```



**SUMMERY**

say somthing

#IMPORTS

In [0]:
CELL_NAME="IMPORTS"
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf
#tf.enable_eager_execution()
from datetime import datetime


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import random
import numpy as np
import os
import re
import unicodedata

#import time


#We recommend you upgrade now or ensure your notebook will continue to use TensorFlow 1.x via the %tensorflow_version 1.x magic: more info.


#GLOBAL VARS

In [0]:
this_time=str(datetime.now())
GLOBAL_epoch=0
DEBUG=False

#LOGGING

In [0]:
######To clean logs
######!rm log*.log

In [295]:
CELL_NAME="START LOG"


log_file="LOG___"+this_time+".txt"
f_logg= open(log_file,"w")
print("logging to file (will be added to mail)",log_file)
  




def _print_log(also_print,*txts):  
  txt=""
  for t in txts:
    txt+=" "+str(t)
  f_logg.write(CELL_NAME+" "+str(datetime.now())+": "+txt+'\n') #TODO ADD TIME!!!!
  if also_print:
    print(*txts)

def print_log(*txts):
  if (DEBUG):
    print(*txts)
  _print_log(False,*txts)


def print_log_screen(*txts):
  _print_log(True,*txts)


def log_flush():
  f_logg.flush()

def close_log():
  f_logg.close()



#DON'T FORGET TO CLOSE THE FILE AT THE END


logging to file (will be added to mail) LOG___2020-05-06 21:49:55.855631.txt


In [296]:

#PAIST HERE TO SEE PROPERLY THE SHOW DIFF PART IN LOGS
text='''MAIN:  (1) ‫ אלא באד'נה . פכיפ לא | إلاّ بإذنه . فكيف لا | إلاّ بإذنه . فكيف لا | 0.0000
MAIN:  (2) ‫ להמ , ולטלבוא וג'והא | لهم , ولطلبوا وجوها[1m[31mً[0m | لهم , ولطلبوا وجوها | 0.0500
MAIN:  (3) ‫ , ודפעהמא אלי מוסי H | , ودفعهما إلى موسى H | , ودفعهما إلى موسى H | 0.0000
MAIN:  LER (label error rate):  0.033400332030791034'''
print(text)





MAIN:  (1) ‫ אלא באד'נה . פכיפ לא | إلاّ بإذنه . فكيف لا | إلاّ بإذنه . فكيف لا | 0.0000
MAIN:  (2) ‫ להמ , ולטלבוא וג'והא | لهم , ولطلبوا وجوهاً | لهم , ولطلبوا وجوها | 0.0500
MAIN:  (3) ‫ , ודפעהמא אלי מוסי H | , ودفعهما إلى موسى H | , ودفعهما إلى موسى H | 0.0000
MAIN:  LER (label error rate):  0.033400332030791034


#Mount Drive

In [297]:
CELL_NAME="MOUNT DRIVE"
print_log_screen("mounting to drive at /gdrive")
from google.colab import drive
drive.mount('/gdrive')


mounting to drive at /gdrive
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


#MODEL PARAMETERS

In [298]:
CELL_NAME="BATCH_SIZE and STATEFUL"

STATEFUL=False
BATCH_SIZE = 128

print_log_screen("set batch size to "+str(BATCH_SIZE))
print_log_screen("STATEFUL: "+str(STATEFUL))

set batch size to 128
STATEFUL: False


# INIT RANDOM

In [0]:
CELL_NAME="RANDOM SEED"

#https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed
def init_random():
  np.random.seed(1)
  tf.compat.v1.set_random_seed(1)


In [0]:
CELL_NAME="RANDOM SEED"
init_random()

#CONSTANTS

In [0]:
arab_nikud=[u"\u0652",u"\u0650", u"\u064F",u"\u064E", ]#sukuun,kasra, Damma,# fatHa
tanween=[u"\u064B", # fatHatayn
         u"\u064C", # Dammatayn
         u"\u064D", ]
shada=u"\u0651"

hamza_on_line=u"\u0621"

LTRchar=u'\u202B'   #align rtl symbole
BLANK="_"


#UTILS

##send mail

In [0]:
CELL_NAME="SEND MAIL"

#NEED TO ALLOW LESS SECURE APPS AT:  
#https://myaccount.google.com/lesssecureapps?utm_source=google-account&utm_medium=web

#Send Alert Email at finish with GMail
##ref: https://webcache.googleusercontent.com/search?q=cache:peuNIUcC5eAJ:https://rohitmidha23.github.io/Colab-Tricks/+&cd=1&hl=en&ct=clnk&gl=il
#https://www.google.com/search?safe=strict&rlz=1C1SQJL_iwIL818IL818&sxsrf=ACYBGNQn05BVmX0bKCQOdxEZsOV8sylztA%3A1568909507810&ei=w6iDXeKYMZLSxgO1qYSICg&q=smtplib.smtp+sendmail+attachment&oq=smtplib.smtp+sendmail+att&gs_l=psy-ab.3.0.33i21j33i160.1435.2378..3438...0.2..0.188.632.0j4......0....1..gws-wiz.......0i71j0j0i22i30.7MbuYV36t10
####how to define app password see: https://kinsta.com/knowledgebase/free-smtp-server/

import smtplib
from os import path
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate

def send_results(subject,description):
  THISTHIS="qczvfrlypitxxsfc"

  server = smtplib.SMTP('smtp.gmail.com', 587)
  #server = smtplib.SMTP('localhost')
  server.starttls()
  server.login("kuti.sulimani@gmail.com", THISTHIS)

  msg = MIMEMultipart()
  msg['From'] = "sender_gmail_here@gmail.com"
  msg['To'] = COMMASPACE.join(["oriterner@gmail.com"])
  msg['Date'] = formatdate(localtime=True)
  msg['Subject'] = subject


  msg.attach(MIMEText(description))
  files=[log_file,"/content/train.png","/content/test.png","/content/accuracys.png","/content/my_log.txt"]  #list of graphs to send or logs....
  for f in files or []:
      if not path.exists(f):
        continue
      with open(f, "rb") as fil:
          part = MIMEApplication(
              fil.read(),
              Name=basename(f)
          )
      # After the file is closed
      part['Content-Disposition'] = 'attachment; filename="%s"' % basename(f)
      msg.attach(part)


  server.sendmail("sender_gmail_here@gmail.com", "oriterner@gmail.com", msg.as_string())
  server.quit()

##plot

In [0]:
CELL_NAME="PLOT"

import matplotlib.pyplot as plt
losses=[1,2,3]
def my_plot_save(data_series,save_name,decor='r--'):
  t = range(0, len(data_series))
  plt.plot(t, data_series, decor)
  plt.savefig(save_name) #"/content/foo.png"
  plt.show()
#my_plot_save(losses,"train.png")

##letter mapping

In [0]:
CELL_NAME="LETTER MAPPING"

tag="'"

additional_letters=".H,?:;[]()!-\" 0123456789"

#"א": "اإآٱأ", with wasla
letter_dict={   #make sure all are here
    "א": "اإآأ",
    "ב":"ب" ,
    "ג":"غ",
    "ג"+tag:"ج",
    "ד":"د",
    "ד"+tag:"ذ",
    "ה":"ه",
    "ה"+tag:"ة",
    "ו":"وؤ",
    "ז":"ز",
    "ח":"ح",
    "ט":"ط",
    "ט"+tag:"ظ",
    "י":"يىئ",
    "כ":"ك",
    "כ"+tag:"خ",
    "ל":"ل",
    "מ":"م",
    "נ":"ن",
    "ס":"س",
    "ע":"ع",
    "פ":"ف",
    "צ":"ص",
    "צ"+tag:"ض",
    "ק":"ق",
    "ר":"ر",
    "ש":"ش",
    "ת":"ت",
    "ת"+tag:"ث",
}

for i in additional_letters:
  letter_dict[i]=i

arab_heb_maping={}
heb_arab_maping={}
for heb,arr in letter_dict.items():
  heb_arab_maping[heb]=arr[0]
  for a in arr:
    arab_heb_maping[a]=heb


print_log("arab_heb_maping",arab_heb_maping)
print_log("length:",len(arab_heb_maping))
print_log("heb_arab_maping",heb_arab_maping)
print_log("length:",len(heb_arab_maping))



CELL_NAME="SIMPLE LETTER MAP"

def simple_letter_map(heb_str): 
  res=[]
  tag="'"
  iterator = iter(range(len(heb_str)))
  for i in iterator:
    if i+1!=len(heb_str) and heb_str[i+1]==tag:     
      if heb_str[i]+tag in heb_arab_maping:
        ar_leter=heb_arab_maping[heb_str[i]+tag]
        res.append(ar_leter)
      else:
        ar_leter=heb_arab_maping[heb_str[i]]
        res.append(ar_leter)
        res.append(tag)
      next(iterator, None)
    else:      
      ar_leter=heb_arab_maping[heb_str[i]]
      res.append(ar_leter)
  return "".join(res)     


def reverse_simple_map(arr_str):
  ja_str=[]
  for c in arr_str:
    if c in arab_heb_maping:
      ja_str.append(arab_heb_maping[c])
    #else:
      #print_log("reverse_simple_map: char ( "+c+" ) not in letter mapping and will be skiped")
      #print_log(arr_str)
  return "".join(ja_str)

#Data

In [0]:
CELL_NAME="DATA PATHS"

hakuzari="/gdrive/My Drive/thesis-data/for_ctc_train22_FRIDBERG3.txt"
haemunot="/gdrive/My Drive/thesis-data/haemunot_vedeot/for_ctc_train22_FRIDBERG5.txt"
kfir_kuzari_test="/gdrive/My Drive/thesis-data/kfir1/kfir_kuzari_test.txt"
kfir_rasag_test="/gdrive/My Drive/thesis-data/kfir1/kfir_rasag_test.txt"


##preprocess sentences

In [0]:
CELL_NAME="HELPERS"

##HELPERS
def print_by_idx_CTC(idx,dict,leng=-1):
     # print_log(len(idx))
      if leng==-1:
       # print_log(len(idx))
        leng=len(idx)
        
      result=""
      for i in idx[:leng]:
        result += dict[i.numpy()]
      return result

# def view_data(data):
#   for i,j,l1,l2 in data.take(3):
#     print_log_screen(LTRchar,undouble_hebrew(print_by_idx_CTC(i[0],inp_lang.idx2char,l1[0]))," | ",print_by_idx_CTC(j[0],targ_lang.idx2char,l2[0]))

def view_data(data):
  for i,j,l1,l2 in data.take(1):
    for t in range(5):
      print_log_screen(LTRchar,undouble_hebrew(print_by_idx_CTC(i[t],inp_lang.idx2char,l1[t]))," | ",print_by_idx_CTC(j[t],targ_lang.idx2char,l2[t]))



#s is a sentence
#dict as a dictionary that maps chars to ints
def vectorize(s,dict):  
  #return [dict[c] for c in s]
  res=[]
  for c in s:
    if c not in dict:
      print_log(LTRchar,"The char ( ", c," ) is not in dict (skipping this char). Full sentences: ")
      print_log(LTRchar+s)
    else:
      res.append(dict[c])  
  return res

def clear_blank(s):
  return s.replace(BLANK,"")


In [0]:
CELL_NAME="PREPROCESS SENTENCES"


def normalize_unicode(s):
    s = s.strip()
    return ''.join(c for c in unicodedata.normalize('NFC', s))
        #if unicodedata.category(c) != 'Mn')


#ARABIC:
def remove_arab_nikud(s):
  return ''.join(c for c in  s  if c not in arab_nikud)

def clear_arab_punctuation(s): 
  return s.replace("،",",").replace("؛",";").replace("؟","?")

def standard_nunization(s):
  return s.replace("ًا","اً")

#CHECK
assert(standard_nunization("بيتًا")=="بيتاً")

#JUDEO-ARABIC
def preprocess_hebrew(w):
    w = normalize_unicode(w.strip())
    w = w.replace('ם','מ').replace('ן','נ').replace('ץ','צ').replace('ף','פ').replace('ך','כ')
    w = w.replace('ֿ',"'") #sometimes instead of tag there is a horizontal line above letter
    return w


def dropout(ja_str,keep):
  res=[]
  for c in ja_str:
    if c==" ":
      res.append(c)
    elif (np.random.binomial(1,keep)):
      res.append(c)
    else:
      res.append(BLANK)
  return "".join(res)


def double_hebrew(w):    
    res=""
    for i in w:
      res+=i
      if not i==" ":  ##THIS 2 LINES IS THE CHANGE THAT WAS ADDED AT THE LAST MINUTE 
        res+=i    
    return res


def undouble_hebrew(s):
  res=""
  words=s.split()
  for w in words:
    for i in range(0,len(w),2):
      res+=w[i]
    res+=' '
  return res.strip()




##languageIndex

In [0]:
CELL_NAME="LANGUAGE INDEX"

# This class creates a char -> index mapping (e.g,. "d" -> 5) and vice-versa 
# (e.g., 5 -> "d") for each language,

#this class takes a corpus of lines (lang) and extract the vocab
# (letters and signs), stores the corpus and the vocab (with revers map)
# it also addes the BLANK symbol to the vocab. (makes sure that BLANK is not in the corpus)
BLANK="_"
class LanguageIndex():
  def __init__(self, allowed_letters):
    self.allowed_letters = allowed_letters
    self.char2idx = {}
    self.idx2char = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for c in self.allowed_letters:
      #for c in phrase:     #for the meantime don't habdle the diatrics in- hebrew (the tag) and hope the ctc will handle...wishfully
        self.vocab.update(c)
      #for c in additional_letters:
      #  self.vocab.update(c)
    self.vocab = sorted(self.vocab)
    print_log("vocab: ",self.vocab)   # maps id (i.e. map index) to char
    
    
    for index, char in enumerate(self.vocab): #reverse map: char to id
      self.char2idx[char] = index
    print_log("len(self.vocab)",len(self.vocab))
    assert(BLANK not in self.char2idx)
    self.char2idx[BLANK] = len(self.vocab)   #add BLANK to reverse map
    print_log("len(self.char2idx)",len(self.char2idx)) #should print successor of privous print
    print_log(self.char2idx[BLANK],BLANK)
    
    
    for char, index in self.char2idx.items():  #this is a map equal to the array vocab, but with BLANK
      self.idx2char[index] = char

In [0]:
CELL_NAME="LANGUAGE INDEX"

inp_lang = LanguageIndex("".join(heb_arab_maping.keys()))
targ_lang = LanguageIndex("".join(arab_heb_maping.keys())+"".join(tanween)+ shada + hamza_on_line)



##FUCNTION FOR GEN DATA

In [0]:
CELL_NAME="LOAD_LINES"

#THIS IS REDUNDANT can add to create dataset function   
def load_lines(input_file=hakuzari):
  print_log_screen("loading text: "+input_file)
  with open(input_file, 'rb') as f:
    text = f.read().decode(encoding='utf-8')
    #text=text.replace('ֿ',"'")   ##allread doing it inside preprocess_hebrew()
    text=text.replace("&nbsp","")
  lines=text.strip().split('\n')
  print_log("first lines:",lines[0:100])
  print_log_screen("len(lines)", len(lines)) # 10923 kuzari 10358 haemunot
  return lines

#lines=load_lines(haemunot)


In [0]:
# Takes a file of <heb, arab> phrases separated by tab
# Return phares pairs in the format: [ENGLISH, SPANISH]
def create_dataset(lines,keep=1):
    word_pairs=[]
    for l in lines:
      heb,arr=l.split('\t')
      heb=clear_blank(clear_arab_punctuation(heb)) #TODO needed?
      heb=preprocess_hebrew(heb)
      heb=dropout(heb,keep)
      heb=double_hebrew(heb)      

      arr=remove_arab_nikud(arr)
      arr=standard_nunization(arr)
      arr=clear_blank(clear_arab_punctuation(arr))
      arr=normalize_unicode(arr) 
      
      word_pairs.append([heb,arr])        
    return word_pairs


load_dataset method

In [0]:
CELL_NAME="DEF LOAD_DATASET"

def max_length(tensor):
    return max(len(t) for t in tensor)


def load_dataset(pairs):
  
    input_tensor = [vectorize(heb,inp_lang.char2idx) for heb, arr in pairs]
    input_lenghts=[len(heb) for heb,arr in pairs]
  
    target_tensor = [vectorize(arr,targ_lang.char2idx) for heb, arr in pairs]
    target_lengths = [len(arr)  for heb,arr in pairs]
  

    print_log()
    print_log(LTRchar,pairs[0])
    print_log(input_lenghts[0])
    print_log(target_lengths[0])
    print_log(input_tensor[0])
    print_log(target_tensor[0])

    # Padding the input and output tensor to the maximum length
    max=max_length(input_tensor)
    if max>70:
      max=70
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max,
                                                                 padding='post',
                                                                  value=inp_lang.char2idx[BLANK])
    max=max_length(target_tensor)
    if max>70:
      max=70    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max,
                                                                  padding='post',
                                                                  value=targ_lang.char2idx[BLANK])
    return input_tensor, target_tensor ,input_lenghts,target_lengths

generate the data tensor

In [0]:
CELL_NAME="DEF GEN DATA"

#TODO split in file!!!
#ref https://stackoverflow.com/questions/42471570/how-to-split-documents-into-training-set-and-test-set
# docs = np.array([
#     'one', 'two', 'three', 'four', 'five',
#     'six', 'seven', 'eight', 'nine', 'ten',
#     ])

# idx = np.hstack((np.ones(7), np.zeros(3))) # generate indices
# np.random.shuffle(idx) # shuffle to make training data and test data random

# train = docs[idx == 1]
# test = docs[idx == 0]

# print(train)
# print(test)


def gen_data(input_tensor, target_tensor,input_lenghts,target_lengths,_test_size=0.2):  
  input_tensor_train, input_tensor_val, \
  target_tensor_train, target_tensor_val, \
  input_lengths_train, input_lengths_val, \
  target_lengths_train, target_lengths_val = train_test_split(input_tensor,
                                                              target_tensor,
                                                              input_lenghts,
                                                              target_lengths, test_size=_test_size)
  print_log(len(input_tensor_train), 
        len(target_tensor_train), 
        len(input_tensor_val), 
        len(target_tensor_val))
  
  BUFFER_SIZE = len(input_tensor_train)
  
  dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, 
                                                target_tensor_train,
                                                input_lengths_train,
                                                target_lengths_train)).shuffle(BUFFER_SIZE,reshuffle_each_iteration=True)


  test_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, 
                                                    target_tensor_val,
                                                    input_lengths_val,
                                                    target_lengths_val)).shuffle(BUFFER_SIZE,reshuffle_each_iteration=False)                                                  
  
  dataset_double=dataset.batch(BATCH_SIZE, drop_remainder=True)

  test_dataset_double=test_dataset.batch(BATCH_SIZE, drop_remainder=True)

  return dataset_double,test_dataset_double



##activate gen data

In [314]:
CELL_NAME="LOAD DATASET"


lines=load_lines(hakuzari)
pairs = create_dataset(lines,1)  
input_tensor, target_tensor ,input_lenghts,target_lengths = load_dataset(pairs)
dataset_double_kuzari,test_dataset_double_kuzari=gen_data(input_tensor, target_tensor,input_lenghts,target_lengths)
print_log_screen("test_dataset_double_kuzari")
view_data(test_dataset_double_kuzari)

lines1=load_lines(haemunot)
pairs1 = create_dataset(lines1,1)  
input_tensor1, target_tensor1 ,input_lenghts1,target_lengths1 = load_dataset(pairs1)
dataset_double_rasag,test_dataset_double_rasag=gen_data(input_tensor1, target_tensor1,input_lenghts1,target_lengths1)
print_log_screen("test_dataset_double_rasag")
view_data(test_dataset_double_rasag)


# def gen kuzari_drop(lines,keep):
#   pairs = create_dataset(lines,0.90)  
#   input_tensor, target_tensor ,input_lenghts,target_lengths = load_dataset(pairs)
#   dataset_double_kuzari,test_dataset_double_kuzari=gen_data(input_tensor, target_tensor,input_lenghts,target_lengths)
  

loading text: /gdrive/My Drive/thesis-data/for_ctc_train22_FRIDBERG3.txt
len(lines) 10923
test_dataset_double_kuzari
‫ ולא ידרכה כונ ולא פסאד  |  ولا يدركه كون ولا فساد
‫ עלי אלחקיקה , פלנא אנ  |  على الحقيقة , فلنا أن
‫ . . . H . לאנה למ יחפט'  |  . . . H . لأنّه لم يحفظ
‫ אנהא לא תג'ד אלא פי אלהנד  |  أنّها لا توجد إلاّ في الهند
‫ ולא ינתהי אלי ג'רצ' מקצוד  |  ولا ينتهي إلى غرض مقصود
loading text: /gdrive/My Drive/thesis-data/haemunot_vedeot/for_ctc_train22_FRIDBERG5.txt
len(lines) 10358
test_dataset_double_rasag
‫ אלמעג'זאת אלתי דפעת אליה  |  المعجزات التي دفعت إليه
‫ תקויה' אלקלוב מנ קבול  |  تقوية القلوب من قبول
‫ . ת'מ ערפנא אנ ג'מיע  |  . ثمّ عرّفنا أنّ جميع
‫ , ואנמא ראוא פעלהא ,  |  , وإنما رأوا فعلها ,
‫ בינ פי אלתוריה אנה ג'על  |  بيّن في التورية أنه جعل


##SYNTHETIC DATA

In [0]:
CELL_NAME="GEN SYNTH"

#TODO edit when time permits

def load_text_for_synth(ibnsina="/gdrive/My Drive/JUDEO-ARAB/ibn_sina_ilhyat.txt"):
  with open(ibnsina, 'rb') as f:
      ibnsina_text = f.read().decode(encoding='utf-8')


  ibnsina_text=clear_arab_punctuation(ibnsina_text) #TODO WHAT ABOUT OTHER ARAB NORMALIZATION???
  #TODO somthing with new line - this indicates new content!!!

  #add space before and after punctuation signs
  ibnsina_text = re.sub(r"([:;?.!,¿])", r" \1 ", ibnsina_text)
  ibnsina_text = re.sub(r'[" "]+', " ", ibnsina_text)    
  
  return ibnsina_text




In [0]:
CELL_NAME="DEF gen_synth_sentences"

def gen_synth_sentences(sina_words):
  #SENTENCE_LIMIT=random.randint(1,50) #this didn't work. try random with range1,10
  SENTENCE_LIMIT=20
  sentences=[]
  char_count=0
  res=[]
  for w in sina_words:
  #  w=w.rstrip(" ").strip(" ")
    char_count+=len(w)+1 #len of word + space afterwards
    res.append(w)
    if char_count>SENTENCE_LIMIT:    
      sentences.append(normalize_unicode(remove_arab_nikud(" ".join(res))))
      res=[]
      char_count=0
    #  SENTENCE_LIMIT=random.randint(1,50)          
  return sentences

# #ACTIVATE
# sentences=gen_synth_sentences(sina_words)
# len(sentences)
# sentences[:5]


In [0]:
CELL_NAME="DEF gen_dropout"

def gen_dropout(sentences,keep=0.90):
  arab_setences=[]
  heb_sentences=[]
  for arr in sentences:
    arab_setences.append(arr)           
    heb=reverse_simple_map(arr)
    heb=dropout(heb,keep)
    heb_sentences.append(double_hebrew(preprocess_hebrew(heb)))

  print_log(heb_sentences[:5]) 
  print_log(arab_setences[:5])

  print_log(len(arab_setences))
  input_tensor_synth, target_tensor_synth, \
  input_lenghts_synth,target_lengths_synth = load_dataset(list(zip(heb_sentences,arab_setences)))
  
  # Show length
  print_log(len(input_tensor_synth), len(target_tensor_synth))
  print_log(len(input_lenghts_synth), len(target_lengths_synth))

  BUFFER_SIZE = len(input_tensor_synth)

  dataset_synth = tf.data.Dataset.from_tensor_slices((input_tensor_synth,
                                                      target_tensor_synth,
                                                      input_lenghts_synth,
                                                      target_lengths_synth)).shuffle(BUFFER_SIZE)


  dataset_double_synt=dataset_synth.batch(BATCH_SIZE, drop_remainder=True)
  return dataset_double_synt



In [0]:
CELL_NAME="DEF gen_dropout_all"

def gen_dropout_all(sntcs1,*sentences,keep=1):
  result_dataset=gen_dropout(sntcs1,keep)
  for sntcs in sentences:
    result_dataset.concatenate(gen_dropout(sntcs,keep))
  return result_dataset.shuffle(1000)

#   BUFFER_SIZE=1000 #TODO get size
#   return dataset_double_synt3.concatenate(dataset_double_kuzari).shuffle(BUFFER_SIZE)  ##TODO: this is without dropout. 
# #view_data(gen_dropout_all())

In [319]:


path0="/gdrive/My Drive/JUDEO-ARAB/ibn_sina_ilhyat.txt"
synth_text=load_text_for_synth(path0)
sentences=gen_synth_sentences(synth_text.split())

path1="/gdrive/My Drive/JUDEO-ARAB/daruri-IR.txt"
synth_text1=load_text_for_synth(path1)
sentences1=gen_synth_sentences(synth_text1.split())


path2="/gdrive/My Drive/JUDEO-ARAB/farabi-tahsil.txt"
synth_text2=load_text_for_synth(path2)
sentences2=gen_synth_sentences(synth_text2.split())

path3="/gdrive/My Drive/JUDEO-ARAB/huruf.txt"
synth_text3=load_text_for_synth(path3)
sentences3=gen_synth_sentences(synth_text3.split())



view_data(gen_dropout_all(sentences,sentences1,sentences2,sentences3,keep=0.9))
print("===")
view_data(gen_dropout_all(sentences,sentences1,sentences2,sentences3,keep=0.9))

‫ לא_ הד'א אלתכ'ציצ אמא תכ_י  |  لأن هذا التخصيص إما تكفي
‫ מנ וא_ד מעינ עלה_ לד'את  |  من واحد معين علة لذات
‫ פאנה למא כאנ הנ_כ אלמבדא  |  فإنه لما كان هناك المبدأ
‫ אלג_יה' ובינ א_צ'רורי ו_ע__פ  |  الغاية وبين الضروري وتعريف
‫ אל_ג'ה אלד'י הו אצוב ואלד'י  |  الوجه الذي هو أصوب والذي
===
‫ _פארק _איר אלחיואנאת  |  يفارق سائر الحيوانات
‫ מוג'__ . ת'מ נקל ד'ל_ __י  |  موجود . ثم نقل ذلك إلى
‫ בינ אלמאהיה' פי אלמרכבאת  |  بين الماهية في المركبات
‫ קל_ל , ופי אל_ת'יפ כת'יר  |  قليل , وفي الكثيف كثير
‫ ג_ר מקדור עליה , ב_ בעצ'  |  غير مقدور عليه , بل بعض


#MODEL

In [0]:
CELL_NAME="BUILD MODEL"

#NETWORK PARAMS
# The embedding dimension
embedding_dim = 8 #256

# Number of RNN units
rnn_units = 1024 #1024


if tf.test.is_gpu_available():
  #rnn = tf.keras.layers.CuDNNGRU
  rnn=tf.compat.v1.keras.layers.CuDNNGRU
  #rnn = tf.keras.layers.LSTM #see https://stackoverflow.com/questions/55761337/module-tensorflow-python-keras-api-v2-keras-layers-has-no-attribute-cudnnlst

else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')
  
#FUNCTION TO BUILD MODEL
def build_model(vocab_size_heb1,vocab_size_ar, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size_heb1, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Bidirectional(rnn(rnn_units,     #I added the bidirectianl myself on top of https://www.tensorflow.org/tutorials/sequences/text_generation. this improve the results because the net can predict end of word, and know when to put OTIOT SOFIOT 
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=STATEFUL)),

   tf.keras.layers.Bidirectional(rnn(rnn_units,     #I added the bidirectianl myself on top of https://www.tensorflow.org/tutorials/sequences/text_generation. this improve the results because the net can predict end of word, and know when to put OTIOT SOFIOT 
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=STATEFUL)),
   tf.keras.layers.Bidirectional(rnn(rnn_units,     #I added the bidirectianl myself on top of https://www.tensorflow.org/tutorials/sequences/text_generation. this improve the results because the net can predict end of word, and know when to put OTIOT SOFIOT 
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=STATEFUL)),
    
  

    tf.keras.layers.Bidirectional(rnn(rnn_units,     #I added the bidirectianl myself on top of https://www.tensorflow.org/tutorials/sequences/text_generation
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=STATEFUL)),
    tf.keras.layers.Dense(vocab_size_ar
                         )
  ])
  return model

In [321]:
CELL_NAME="BUILD MODEL1"
def rebuild():
  #BUILD MODEL
  model = build_model(
    vocab_size_ar = len(targ_lang.char2idx),
    vocab_size_heb1 = len(inp_lang.char2idx),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    )

  model.summary()
  return model
model=rebuild()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (128, None, 8)            384       
_________________________________________________________________
bidirectional_36 (Bidirectio (128, None, 2048)         6352896   
_________________________________________________________________
bidirectional_37 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
bidirectional_38 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
bidirectional_39 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
dense_9 (Dense)              (128, None, 65)           133185    
Total params: 63,146,433
Trainable params: 63,146,433
Non-trainable params: 0
__________________________________________

##CHECKPOINTS

In [0]:
CELL_NAME="DEFINE CHECKPOINT"

checkpoint_path='/gdrive/My Drive/checkpoints/'+this_time+"/ckpt"
def save_checkpoint(massage):
  print_log_screen("saving checkpoing at "+checkpoint_path)
  model.save_weights(checkpoint_path)
  f_check= open(checkpoint_path+".txt","a+")  
  f_check.write("saving chekcpoing at epoch"+ str(GLOBAL_epoch) +'\n')
  f_check.write(massage+'\n')
  f_check.close()
  
def load_checkpoint(checkpoint_path=checkpoint_path):
  print_log_screen("loading checkpoing from "+checkpoint_path)
  model1=rebuild()
  model1.load_weights(checkpoint_path)
  return model1


#TESTING FUNCTIONS

In [0]:
CELL_NAME="DEF show diff"
from termcolor import colored
import difflib

def show_diff(t1,t2,col):
    """Unify operations between two compared strings
seqm is a difflib.SequenceMatcher instance whose a & b are strings"""
    seqm= difflib.SequenceMatcher(None,t1,t2)   
    output1=[]
    output2= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        
        if opcode == 'equal':            
            output1.append(seqm.a[a0:a1])
            output2.append(seqm.b[b0:b1])
        elif opcode == 'insert':            
            output2.append(colored(seqm.b[b0:b1],col,attrs=['bold']))
        elif opcode == 'delete':
            output1.append(colored(seqm.a[a0:a1],col,attrs=['bold']))
        elif opcode == 'replace':            
            output1.append(colored(seqm.a[a0:a1],col,attrs=['bold']))
            output2.append(colored(seqm.b[b0:b1],col,attrs=['bold']))
        else:
            raise RuntimeError("unexpected opcode")
    return ''.join(output1),''.join(output2)

# #USEAGE:
# s1="لامة النصارى واستحقو" 
# s2="لأمّة النصارى واستحقوّا"
# a,b=show_diff(s1,s2,'blue')
# #print_log("".join(b))
# print_log(a,"|",b)



## forward run each letter seperatly

In [0]:
CELL_NAME="TEST single letters"
def test__CTC_letters(): 
  total_accuracy=0
  total_examples=0

  all_heb_letters=inp_lang.vocab
  
  num_of_letters=len(inp_lang.vocab)
  num_of_arab_letters=len(targ_lang.vocab)

#  print_log(num_of_letters)
  letters_as_int=[]
  tag=inp_lang.char2idx["'"]
  for t in range(num_of_letters):
    letters_as_int.append([t]*2)
  for t in range(BATCH_SIZE-num_of_letters):
    letters_as_int.append([0,0])    

  letters_tensor=tf.convert_to_tensor(letters_as_int)
  
  predict_ltrs=model(letters_tensor)
  
  
  for jj in range(num_of_letters):
      print("candidate:***({0})***".format(inp_lang.vocab[jj]))
      
      pred_distr=predict_ltrs[jj][1]
      pred_distr=tf.nn.softmax(pred_distr)
      for ii in range(num_of_arab_letters):
       print("{0:.3f}({1})  ".format(pred_distr[ii],targ_lang.vocab[ii]),end = '')
      print("{0:.3f} <blank>".format(pred_distr[num_of_arab_letters]))
      maximum=tf.argmax(pred_distr).numpy()
      max_score=tf.math.reduce_max(pred_distr).numpy()
      if (maximum<num_of_arab_letters):
        print("prediction***({0})***{1:.3f}".format(targ_lang.vocab[maximum],max_score))
      else:
        print("####max is the blank symbole")
      print_log('-'*10)
  
#test__CTC_letters()

##forward run a sentence (with top n beam search results)

In [0]:
CELL_NAME="DEF test__CTC_word_multiline"
#TODO edit code when time permits

def test__CTC_word_multiline(word_str,num_of_paths=5,_BATCH_SIZE=BATCH_SIZE,print_deteils=False): 
  lines=word_str.split('\n')
  num_of_lines=len(lines)
  assert(num_of_lines<=BATCH_SIZE)

  inputs=[]
  inputs_len=[]
  for l in lines:
    l = preprocess_hebrew(l)
    l = double_hebrew(l)
    v=vectorize(l,inp_lang.char2idx)
    inputs.append(v)
    inputs_len.append(len(v))
  #max_len=max_len(inputs)
  #PADD HORIZANTALY
  for i in range(BATCH_SIZE-num_of_lines):
    inputs.append([0])
    inputs_len.append(1)



  res=[]

  #PADDDDDD
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, 
                                                         maxlen=max_length(inputs),
                                                         padding='post',
                                                         value=inp_lang.char2idx[BLANK])
    
  inputs=tf.convert_to_tensor(inputs)
    
  predict_ltrs=model(inputs)
  #print_log(predict_ltrs.shape)
  
  
  #inputs_len=[num_of_letters]*_BATCH_SIZE
  inputs=tf.transpose(predict_ltrs,perm=[1,0,2]) #[max_time, batch_size, num_classes]
  
  decoded, log_probabilities=tf.nn.ctc_beam_search_decoder(
                      inputs,
                      inputs_len,top_paths=num_of_paths) 
  # if print_deteils:
  #   print_log('#####deteils')
  # for jj in range(num_of_letters):
  #     if print_deteils:
  #       print_log(word_str[jj])
      
  #     pred_distr=predict_ltrs[0][jj]
  #     pred_distr=tf.nn.softmax(pred_distr)
  #     if print_deteils:
  #       for ii in range(num_of_arab_letters):
  #         print_log("{0:.3f}".format(pred_distr[ii]),targ_lang.vocab[ii])
  #       print_log("{0:.3f} <blank>".format(pred_distr[num_of_arab_letters]))
  #       print_log()
  #     maximum=tf.argmax(predict_ltrs[0][jj]).numpy()
  #     if print_deteils:
  #       print_log(maximum)
  #     if (maximum<42):
  #       if print_deteils:
  #         print_log(targ_lang.vocab[maximum])
  #       res.append(targ_lang.vocab[maximum])
  #     else:
  #       if print_deteils:
  #         print_log("####max is the blank symbole")
  #       res.append("-")
  #     if print_deteils:
  #       print_log('-'*10)
  # print_log(word_str)
  #print_log("ARGMAX PREDICTION: ","".join(res))  
  total_res=""
  for t in range(num_of_lines):
    print_log(lines[t])
    for i in reversed(range(num_of_paths)):
      dense=tf.sparse.to_dense(decoded[i])   
      prediction=print_by_idx_CTC(dense[t],targ_lang.idx2char)
      print_log("#####PREDICTION"+str(i)+": ",prediction)
    total_res+='\n'+prediction
  return total_res
  
#IMPORTED OBSERV: it seems that ctc decodes make much more than just argmax over sequences see https://towardsdatascience.com/beam-search-decoding-in-ctc-trained-neural-networks-5a889a3d85a7
###  ا-ل-ط-ب-ي-ع-ي-ة  و-ك-ا-ل-اا   -م---ع-ع-ا-ا   ---> الطبيعية كمالاً مستعدّاً


# lines='''כמאלא
# כמאלא'''

# print_log_screen(test__CTC_word_multiline(lines,3,BATCH_SIZE))

##TEST KFIR

In [0]:
CELL_NAME="DEF TEST KFIR"
import editdistance
#TODO edit code when time permits

def test__CTC_word_multiline_kfir(word_str,targ_str,indexes=None,num_of_paths=1,_BATCH_SIZE=BATCH_SIZE,SHOW_PRINT=False): 
  lines=word_str.split('\n')
  targ_lines=targ_str.split('\n')
  num_of_lines=len(lines)
  num_of_letters= 0
  assert(num_of_lines==len(targ_lines))
  assert(num_of_lines<=BATCH_SIZE)

  inputs=[]
  inputs_len=[]
  for l in lines:
    l = preprocess_hebrew(l)
    #l = double_hebrew(l)
    v=vectorize(l,inp_lang.char2idx)
    inputs.append(v)
    inputs_len.append(len(v))
  #max_len=max_len(inputs)
  #PADD HORIZANTALY
  for i in range(BATCH_SIZE-num_of_lines):
    inputs.append([0])
    inputs_len.append(1)



  res=[]
  #PADDDDDD
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, 
                                                         maxlen=max_length(inputs),
                                                         padding='post',
                                                         value=inp_lang.char2idx[BLANK])
    
  inputs=tf.convert_to_tensor(inputs)
    
  predict_ltrs=model(inputs)
  
  
  
  
  inputs=tf.transpose(predict_ltrs,perm=[1,0,2]) #[max_time, batch_size, num_classes]
  
  
  decoded, log_probabilities=tf.nn.ctc_beam_search_decoder(
                      inputs,
                      inputs_len,top_paths=num_of_paths) 
  
  total_res=[]
  total_edit_dist=0
  total_normalized_edit_dist=0
  line_counter=1
  for t in range(num_of_lines):
    real=targ_lines[t]
    for i in reversed(range(num_of_paths)):
      dense=tf.sparse.to_dense(decoded[i])   
      prediction=print_by_idx_CTC(dense[t],targ_lang.idx2char).strip() 
      # print_log("#####PREDICTION"+str(i)+": ",prediction)
      total_res.append(prediction)
    if indexes:
      real=real.split()[indexes[t]]
      prediction=prediction.split()[indexes[t]]
    ed_dist=editdistance.eval(real, prediction)
    #print(real, len(real))
    num_of_letters+=len(real)

    normalized_ed_dist=ed_dist/len(real)
    real,prediction=show_diff(real,prediction,'red')
    if SHOW_PRINT:
      print_log_screen("({0})".format(line_counter),LTRchar,undouble_hebrew(lines[t]),"|",real,"|",prediction,"|","{0:.4f}".format(ed_dist))
    line_counter+=1
    total_normalized_edit_dist+=normalized_ed_dist
    total_edit_dist+=ed_dist
  return total_res,total_normalized_edit_dist,num_of_lines,total_edit_dist,num_of_letters
  
#IMPORTED OBSERV: it seems that ctc decodes make much more than just argmax over sequences see https://towardsdatascience.com/beam-search-decoding-in-ctc-trained-neural-networks-5a889a3d85a7
###  ا-ل-ط-ب-ي-ع-ي-ة  و-ك-ا-ل-اا   -م---ع-ع-ا-ا   ---> الطبيعية كمالاً مستعدّاً


lines='''יכ'אלף
עלי
הד'ה
אלאמאנה
ולא'''

lines='''ייככ''אאללףף
עעלליי
ההדד''הה
אאללאאממאאננהה
ווללאא'''

lines='''ייככ''אאללףף
אאללאאממאאננהה ששללווםם'''

# targ_lines='''يخالف
# على
# هذه
# الأمانة
# ولا'''

targ_lines='''يخالف
الأمانة الأمانة'''
indexes=[0,1]
#indexes=None

#test__CTC_word_multiline_kfir(lines,targ_lines,indexes,SHOW_PRINT=True)

In [0]:
CELL_NAME="DEF test_kfir1"

#data_path options:
# kfir_kuzari_test
# kfir_rasag_test

#replace_GAIN - replace JAIN with GIMEL


def test_kfir(data_path,replace_GAIN=False,SHOW_PRINT=False,index_path=None):

  lines=load_lines(data_path)
  pairs = create_dataset(lines)
  if index_path:
    indexes=[]
    f_indexes=open(index_path,'r')
    ind_lines=f_indexes.readlines()
    assert(len(ind_lines)==len(lines))
    for il in ind_lines:
      indexes.append(int(il))
    f_indexes.close()
   # print(indexes)
  if replace_GAIN:
    hebrew_lines=[heb.replace("גג","גג''").replace("גג''''","גג") for heb, arr in pairs]
  else:
    hebrew_lines=[heb for heb, arr in pairs]
  arab_lines=[arr for heb, arr in pairs]

  num_of_lines=len(pairs)
  index=0
  total_num_of_examples=0
  total_num_of_letters=0
  total_sum_e_d_normalized=0
  total_sum_e_d=0
  while index<=num_of_lines:
    batch_hebrew=hebrew_lines[index:index+BATCH_SIZE]
    batch_arab=arab_lines[index:index+BATCH_SIZE]
    if index_path:
      batch_indexes=indexes[index:index+BATCH_SIZE]
    else:
      batch_indexes=None
    _,sum_of_e_d_normalized,num_of_examples,sum_of_e_d,num_of_letters=test__CTC_word_multiline_kfir('\n'.join(batch_hebrew),'\n'.join(batch_arab),batch_indexes,SHOW_PRINT=SHOW_PRINT)
    if SHOW_PRINT:
      print_log_screen("BATCH (sum_of_e_d_normalized,num_of_examples): ",sum_of_e_d_normalized,num_of_examples)
      print_log_screen("BATCH (sum_of_e_d,num_of_letters): ",sum_of_e_d,num_of_letters)
    total_num_of_examples+=num_of_examples
    total_num_of_letters+=num_of_letters
    total_sum_e_d+=sum_of_e_d
    total_sum_e_d_normalized+=sum_of_e_d_normalized
    index+=BATCH_SIZE

  print_log_screen("#examples:",total_num_of_examples,", accuracy:",1-total_sum_e_d_normalized/total_num_of_examples)
  print_log_screen("#letters:",total_num_of_letters,", accuracy1:",1-total_sum_e_d/total_num_of_letters)
  return total_sum_e_d_normalized/total_num_of_examples,total_sum_e_d/total_num_of_letters

# test_kfir(kfir_kuzari_test,SHOW_PRINT=True)

# test_kfir(kfir_rasag_test,SHOW_PRINT=True)



In [328]:
CELL_NAME="FOR NACHUM - CONTEXT ON KFIR"

import re
from termcolor import colored, cprint



#adjust test kfir to comp edits only for index word!
DO_EMUNOT=True

if DO_EMUNOT:
  file_all = open(haemunot, "r")
  file = open(kfir_rasag_test, "r")
else:
  file_all = open(hakuzari, "r")
  file = open(kfir_kuzari_test, "r")

arabic_lines=[]
for l in file_all:
  arabic_lines.append(remove_arab_nikud(standard_nunization(normalize_unicode(l))))

greped=[]
for l in file:
  greped.append(remove_arab_nikud(standard_nunization(normalize_unicode(l))))


def grep(reg,ja,full_f,single_f,index_f):
  #file_all = open(haemunot, "r")
  for line in arabic_lines:
      line=remove_arab_nikud(line)     
      JA_line,arab_line=line.split('\t')
      if re.search(reg, arab_line):          
          arab_words=arab_line.split()
          JA_words=JA_line.split()
          reg_ind=arab_words.index(reg.strip(" $^"))
          if (not JA_words[reg_ind]==ja):
            #TODO search for other option in text....
             print("ja:"+ja+ "JA_words[reg_ind]:" +JA_words[reg_ind])
             print("JA_WORDS[reg_ind] (my version) will be outputed to both tests (with and without context)")
          assert(len(JA_words)==len(arab_words)) #MAYBE NOT ALWAYS THE CASE!!!          
          arab_text=[]
          JA_text=[]
          for i in range(len(arab_words)):
            if i==reg_ind:
              arab_text.append(colored(arab_words[i],'red'))
              JA_text.append(colored(JA_words[i],'red'))
            else:
              arab_text.append(arab_words[i]) 
              JA_text.append(JA_words[i])
          full_line=" ".join(JA_text)+'\t'+" ".join(arab_text)+'\t'+str(reg_ind)
          full_f.write(" ".join(JA_words)+'\t'+" ".join(arab_words)+'\n')
          index_f.write(str(reg_ind)+'\n')
          single_words=JA_words[reg_ind]+'\t'+arab_words[reg_ind]
          single_f.write(single_words+'\n')
          return full_line
  return
          

not_found_counter=0

f1=open("with_context.txt",'w+')
f2=open("no_context.txt",'w+')
f3=open("context_idexes.txt",'w+')

for l in greped:  
  arab=l.split('\t')[1].strip()
  ja=l.split('\t')[0].strip()
 # print(ja)
  if DO_EMUNOT:
    ja=ja.replace("ג","ג'").replace("ג''","ג")    ##only for kuzari - change to regular (or won't grep)
 # print(ja)
  arab=' '+arab+' ' 
  found=grep(arab,ja,f1,f2,f3)
 # print("grep: "+arab)
  if not found:
    found=grep("^"+arab.lstrip(),ja,f1,f2,f3)
    if not found:
      found=grep(arab.rstrip()+"$",ja,f1,f2,f3)
      if not found:
        not_found_counter+=1
  if not found:
    print("not found:"+arab)
  else:
    print(found)
f1.close()
f2.close()
f3.close()
print("#not founds:"+str(not_found_counter))

פי דאר אלדניא . ואבתדי	في دار الدنيا . وابتدئ	1
נפענא עלי אלג'זא אצ'עאף	نفعنا على الجزاء أضعاف	2
. וקבל ד'לך מא ראי אן	. وقبل ذلك ما رأى أن	1
מת'ל ד'לך בקולה : H H	مثل ذلك بقوله : H H	1
אמא עלי את'ר מא אפתתחנא	أمّا على إثر ما افتتحنا	3
ואחד מן ראי נפסה פי אכ'טארה	واحد من رأى نفسه في إخطاره	2
H H H . וארי אן אג'על	H H H . وأرى أن أجعل	5
יפרק בין רוחה וג'סמה	يفرّق بين روحه وجسمه	0
אלוקוף בין ידיה פלא יערפה	الوقوف بين يديه فلا يعرفه	1
וטארת רוחה כמא קאל H	وطارت روحه كما قال H	1
יפרק בין רוחה וג'סמה	يفرّق بين روحه وجسمه	3
חתי יתם וצולהם אלי אלמטלוב	حتّى يتمّ وصولهم إلى المطلوب	3
אי וקת טלעתה כד'אך ואשד	أي وقت طلعته كذاك وأشد	1
אלי וקת אסתכמאל אלנפוס	إلى وقت استكمال النفوس	2
פי אלנפוס פמאזג'תהא פצאר	في النفوس فمازجتها فصار	1
וכל נאר חתי תלחק בתראב	وكل نار حتى تلحق بتراب	2
מן גירכם יג'מעהא אלי	من غيركم يجمعها إلى	2
. ואכת'ר מן אלג'מיע מן	. وأكثر من الجميع من	3
אמא עלי את'ר מא אפתתחנא	أمّا على إثر ما افتتحنا	1
אמא עלי את'ר מא אפתתחנא	أمّا على إثر ما افتتحنا	3
. פאד' קד בינת הד'ה אלמקדמה	. فإذ

In [329]:
CELL_NAME="FOR NACHUM - CONTEXT ON KFIR 1"


PRINT_DETAILED=True



model=load_checkpoint('/gdrive/My Drive/checkpoints/2020-05-04 12:32:58.607342/ckpt')  #THIS IS THE FIRST ACTIVATIION, BY WHICH I PRODUCED THE EXCEL FOR NACHUM

#model=load_checkpoint('/gdrive/My Drive/checkpoints/2020-05-05 13:01:42.636420/ckpt')
#if STATEFUL:
model.reset_states()
test_kfir("with_context.txt",replace_GAIN=True,SHOW_PRINT=PRINT_DETAILED,index_path="context_idexes.txt")

print("="*200)
#if STATEFUL:
model.reset_states()

test_kfir("no_context.txt",replace_GAIN=True,SHOW_PRINT=PRINT_DETAILED)

# print("="*200)
# model.reset_states()

# test_kfir("no_context.txt",True)

# print("="*200)
# model.reset_states()

# test_kfir("no_context.txt",True)

#model.reset_states()
#test_kfir(kfir_rasag_test,True)

loading checkpoing from /gdrive/My Drive/checkpoints/2020-05-04 12:32:58.607342/ckpt
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (128, None, 8)            384       
_________________________________________________________________
bidirectional_40 (Bidirectio (128, None, 2048)         6352896   
_________________________________________________________________
bidirectional_41 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
bidirectional_42 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
bidirectional_43 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
dense_10 (Dense)             (128, None, 65)           133185    
Total params: 63,146,433
Trainable

(0.06822157434402333, 0.07065217391304347)

##test baseline

In [330]:
CELL_NAME="TEST_BASELINE"
import editdistance
def test_loss_baseline(this_dataset=test_dataset_double_kuzari,only_first=True,limit=False):
  num_of_paths=1
  total_loss=0
  total_accuracy=0
  total_examples=0
  line_counter=1
  if limit:
    this_dataset=this_dataset.take(limit)
  for input_example_batch, target_example_batch, inputs_len,targets_len in this_dataset:
			
          for i in range(BATCH_SIZE):
                heb_input=print_by_idx_CTC(input_example_batch[i],inp_lang.idx2char)

                heb_input=undouble_hebrew(heb_input).strip(BLANK)
                prediction=simple_letter_map(heb_input)
                real=print_by_idx_CTC(target_example_batch[i],targ_lang.idx2char,targets_len[i].numpy()).strip(BLANK)  
                accuracy=editdistance.eval(real, prediction)
                accuracy/=len(real)
                total_accuracy+=accuracy
                if only_first and i!=0:
                    continue
                real,prediction=show_diff(real,prediction,'red')
                print_log_screen("({0})".format(line_counter),LTRchar,heb_input.strip(BLANK),"|",real,"|",prediction,"|","{0:.4f}".format(accuracy))
                line_counter+=1
          total_examples+=BATCH_SIZE
							 
  if total_accuracy!=0:
    total_accuracy/=total_examples
    print_log_screen("baseline accuracy: ",1-total_accuracy)
												
  return 1-total_accuracy
test_loss_baseline(limit=3)
# print_log_screen("KUZARI TEST")
# test_loss_baseline(test_dataset_double_kuzari)
# print_log_screen("RASAG TEST")
# test_loss_baseline(test_dataset_double_rasag)

(1) ‫ ולא ידרכה כונ ולא פסאד | ولا يدركه كون ولا فساد | ولا يدركه كون ولا فساد | 0.0000
(2) ‫ אלבאטנ ואלמעני גיר אלמג'תסמ | الباطن والمعنى غير المتجسّم | الباطن والمعني غير المجتسم | 0.1481
(3) ‫ . לאנ אלואחד ענדנא מא | . لأنّ الواحد عندنا ما | . لان الواحد عندنا ما | 0.0909
baseline accuracy:  0.9077731190508334


0.9077731190508334

##TEST BASELINE KFIR

In [331]:
CELL_NAME="DEF TEST_BASELINE_KFIR"

import editdistance

#data_path options:
# kfir_kuzari_test
# kfir_rasag_test

#replace_GAIN - replace JAIN with GIMEL

def test_loss_baseline_kfir(data_path,replace_GAIN=False,SHOW_PRINT=False):
  lines=load_lines(data_path) #THIS IS DOUBLED ALLREADY
  pairs = create_dataset(lines)  
  
  if replace_GAIN:
    print_log_screen("replaceing gimel with jain to match my train convention")
    hebrew_lines=[heb.replace("גג","גג''").replace("גג''''","גג") for heb, arr in pairs] #ייננבבגג''יי
  else:
    hebrew_lines=[heb for heb, arr in pairs]
  arab_lines=[arr for heb, arr in pairs]
  total_examples=len(pairs)
  total_loss=0
  sum_of_e_dist=0
  num_of_letters=0
  for l in arab_lines:
    num_of_letters+=len(l)
  total_accuracy=0
  line_counter=1
  for heb_input,real in zip(hebrew_lines,arab_lines):
                heb_input=undouble_hebrew(heb_input).strip(BLANK)
                prediction=simple_letter_map(heb_input)                
                accuracy=editdistance.eval(real, prediction)
                normalized_accuracy=accuracy/len(real)
                total_accuracy+=normalized_accuracy
                real,prediction=show_diff(real,prediction,'red')
                sum_of_e_dist+=accuracy
                if SHOW_PRINT:
                  print_log_screen("({0})".format(line_counter),LTRchar,heb_input.strip(BLANK),"|",real,"|",prediction,"|",str(accuracy))
                line_counter+=1
							   
  total_accuracy/=total_examples
  sum_of_e_dist/num_of_letters
  print_log_screen(data_path ,"accuracy: ",1-total_accuracy)
  print_log_screen(data_path ,"accuracy1: ",1-sum_of_e_dist/num_of_letters)
												
  return 0,total_accuracy,sum_of_e_dist/num_of_letters
#test_loss_baseline(limit=3)
#test_loss_baseline_kfir(kfir_kuzari_test)
test_loss_baseline_kfir(kfir_kuzari_test,False,False)
#test_loss_baseline_kfir(kfir_rasag_test)
test_loss_baseline_kfir(kfir_rasag_test,False,False)

loading text: /gdrive/My Drive/thesis-data/kfir1/kfir_kuzari_test.txt
len(lines) 500
/gdrive/My Drive/thesis-data/kfir1/kfir_kuzari_test.txt accuracy:  0.8769578643578643
/gdrive/My Drive/thesis-data/kfir1/kfir_kuzari_test.txt accuracy1:  0.8774193548387097
loading text: /gdrive/My Drive/thesis-data/kfir1/kfir_rasag_test.txt
len(lines) 50
/gdrive/My Drive/thesis-data/kfir1/kfir_rasag_test.txt accuracy:  0.845952380952381
/gdrive/My Drive/thesis-data/kfir1/kfir_rasag_test.txt accuracy1:  0.8489583333333334


(0, 0.15404761904761904, 0.15104166666666666)

##test loss

In [0]:
CELL_NAME="DEF general TEST_LOSS"

import editdistance
def test_loss(this_dataset=test_dataset_double_kuzari,only_first=True,limit=False):
  num_of_paths=1
  total_loss=0
  total_accuracy=0
  total_examples=0
  line_counter=1
  if limit:
    this_dataset=this_dataset.take(limit)
  for input_example_batch, target_example_batch, inputs_len,targets_len in this_dataset:
          predictions = model(input_example_batch)                 
          logits=tf.transpose(predictions,perm=[1,0,2])    
          #loss=tf.nn.ctc_loss_v2(target_example_batch,logits, targets_len,inputs_len,blank_index=targ_lang.char2idx[BLANK])
          loss=tf.nn.ctc_loss(target_example_batch,logits, targets_len,inputs_len,blank_index=targ_lang.char2idx[BLANK])
          cost = tf.reduce_mean(loss)
          total_loss+=cost 
          
          
          #decoded, log_probabilities=tf.nn.ctc_beam_search_decoder_v2(
          decoded, log_probabilities=tf.nn.ctc_beam_search_decoder(
                      logits,
                      inputs_len,top_paths=num_of_paths) 
          dense=tf.sparse.to_dense(decoded[0])
            
          for i in range(BATCH_SIZE):
                heb_input=print_by_idx_CTC(input_example_batch[i],inp_lang.idx2char)

                heb_input=undouble_hebrew(heb_input).strip(BLANK)
                prediction=print_by_idx_CTC(dense[i],targ_lang.idx2char).strip()
                real=print_by_idx_CTC(target_example_batch[i],targ_lang.idx2char,targets_len[i].numpy()).strip(BLANK)  
                accuracy=editdistance.eval(real, prediction)
                accuracy/=len(real)
                total_accuracy+=accuracy
                if only_first and i!=0: 
                  continue
                real,prediction=show_diff(real,prediction,'red')
                print_log_screen("({0})".format(line_counter),LTRchar,heb_input.strip(BLANK),"|",real,"|",prediction,"|","{0:.4f}".format(accuracy))
                line_counter+=1

          total_examples+=BATCH_SIZE
  #total_loss/=total_examples
  if total_accuracy!=0:
    total_accuracy/=total_examples
    print_log_screen("LER (label error rate): ",total_accuracy)
#  print_log("total_test loss: ",total_loss.numpy())
  return total_loss.numpy(),total_accuracy

#test_loss(limit=3)

##test guide perplex

In [333]:
CELL_NAME="GUIDE TEXT"
#NOTICE:there's a mix up compared to the arab translitartaion by attai in the 5 6 raw mark here in brackets

###TODO : change hebrew insertion to "H"

#THIS IS THE ORIGNAL FROM THE GNIZA WEBSITE
guid_text='''כנת איהא אלתלמיד' אלעזיז עברית-ר' עברית-יוסף עברית-ש"צ עברית-ב"ר 
עברית-יהודה עברית-נ"ע למא מת'לת ענדי וקצדת
 מן אקאצי אלבלאד ללקראה עלי , עט'ם שאנך ענדי לשדהֿ חרצך עלי 
 אלטלב ולמא ראיתה פי אשעארך מן שדה' אלאשתיאק ללאמור אלנט'ריה וכאן ד'לך מנד' וצלתני רסאילך ומקאמאתך מן
אלאסכנדריה קבל אן אמתחן
תצורך וקלת לעל שוקה אקוי מן אדראכה פלמא קראת עלי מא קד
קראתה מן עלם אלהיאה ומא תקדם לך ממא לא בד מנה תוטיה להא מן אלתעאלים 
זדת בך גבטה לג'ודה' ד'הנך וסרעה' תצורך וראית שוקך ללתעאלים 
עט'ימא פתרכתך ללארתיאץ' פיהא לעלמי במאלך.   
פלמא קראת עלי מא קד קראתה מן צנאעה' אלמנטק תעלקת אמאלי בך 
וראיתך אהלא לתכשף לך אסראר אלכתב אלנבויה חתי תטלע מנהא עלי מא ינבגי 
אן יטלע עליה אלכאמלון פאכ'ד'ת אן אלוח לך תלויחאת ואשיר לך באשאראת
פראיתך תטלב מני אלאזדיאד וסמתני אן אבין לך אשיא מן אלאמור'''

import re
guid_text=re.sub(r'עברית-[^\s]+', 'H',guid_text)
print(guid_text)

#AND THIS IS FROM THE SECOND PAGE ON (in attai book)
#      אלאלאהיה ואן אכ'ברך בהד'ה מקאצד
# אלמתכלמין והל תלך אלטרק ברהאניה ואן לם תכן פמן אי צנאעה הי
# וראיתך קד שדות שיא מן ד'לך עלי גירי ואנת חאיר קד בדתך אלדהשה
# ונפסך אלשריפה תטאלבך למצא דברי חפץ פלם אזל אדפעך ען ד'לך
# ואמרך אן תאכ'ד' אלאשיא עלי תרתיב קצדא מני אן יצח לך אלחק
# בטרקה לא אן יקע אליקין באלערץ' ולם אמתנע טאל אג'תמאעך בי אד'א
# מא ד'כר עברית-פסוק או נץ מן נצוץ אלחכמים פיה תנביה עלי מעני גריב מן
# תביין ד'לך לך . פלמא קדר אללה באלאפתראק ותוג'הת אלי חית' תוג'הת
# את'ארת מני תלך אלאג'תמאעאת עזימה קד כאנת פתרת וחרכתני גיבתך
# לוצ'ע הד'ה אלמקאלה אלתי וצ'עתהא לך ולאמת'אלך וקלילא מא הם
# וג'עלתהא פצולא מנת'ורה וכל מא אנכתב מנהא פהו יצלך אולא אולא
# חית' כנת ואנת סאלם'''

כנת איהא אלתלמיד' אלעזיז H H H H 
H H למא מת'לת ענדי וקצדת
 מן אקאצי אלבלאד ללקראה עלי , עט'ם שאנך ענדי לשדהֿ חרצך עלי 
 אלטלב ולמא ראיתה פי אשעארך מן שדה' אלאשתיאק ללאמור אלנט'ריה וכאן ד'לך מנד' וצלתני רסאילך ומקאמאתך מן
אלאסכנדריה קבל אן אמתחן
תצורך וקלת לעל שוקה אקוי מן אדראכה פלמא קראת עלי מא קד
קראתה מן עלם אלהיאה ומא תקדם לך ממא לא בד מנה תוטיה להא מן אלתעאלים 
זדת בך גבטה לג'ודה' ד'הנך וסרעה' תצורך וראית שוקך ללתעאלים 
עט'ימא פתרכתך ללארתיאץ' פיהא לעלמי במאלך.   
פלמא קראת עלי מא קד קראתה מן צנאעה' אלמנטק תעלקת אמאלי בך 
וראיתך אהלא לתכשף לך אסראר אלכתב אלנבויה חתי תטלע מנהא עלי מא ינבגי 
אן יטלע עליה אלכאמלון פאכ'ד'ת אן אלוח לך תלויחאת ואשיר לך באשאראת
פראיתך תטלב מני אלאזדיאד וסמתני אן אבין לך אשיא מן אלאמור


In [334]:
CELL_NAME="DEF TEST_GUIDE"

def test_guide(limit=1000000,num_of_paths=5):
  return test__CTC_word_multiline(guid_text,num_of_paths,BATCH_SIZE)
print_log_screen(test_guide())


كنت أيّها التلميذ العزيز H H H H                                                                 
H H لما مثلت عندي وقصدت                                                                          
من أقاصى البلاد للقراءة على , عظم شأنك عندي لشدة حرصك على                                        
الطلب ولمّا رأيته في أشعارك من شدة الاشتياق للأمور النظرية وكان ذلك منذ وصلتني رسائلك ومقاماتك من
الأسكندرية قبل أن امتحن                                                                          
تصورك وقلت لعل شوقه أقوى من إدراكه فلمّا قرأت على ما قد                                          
قراته من علم الهيئة وما تقدّم لك ممّا لا بدّ منه توطئة لها من التعاليم                           
زدت بك غبطة لجودة ذهنك وسرعة تصورك ورأيت شوقك للتعاليم                                           
عظيماً فتركتك للارتياض فيها لعلميّ بمالك                                                         
فلما قرأت على ما قد قراته من صناعة المنطق تعلّقت أمالى بك                                        
ورأيتك أهلاً لتكشف 

#TRAIN

##pre-train letters

In [0]:
CELL_NAME="DEF pretrain_letters"
#train only non-tag letters with cross_entropy

import random


LEN=10

def pretrain_letters(EPOCHS=10000,_BATCH_SIZE=BATCH_SIZE):
  
  for epoch in range(EPOCHS):
    total_loss=0
    if STATEFUL:
      hidden = model.reset_states()  #needed?
    for batch_n in range(30):
        inp=[]
        target=[]
        for i in range(_BATCH_SIZE):
          #draw hebrew letter with tag or not. translate to ints   ###SHOULD USE THE DICT #arab_heb_maping
          heb_res=[]
          arab_res=[]
          for jj in range(LEN):
            choosen_arr=random.choice(list(arab_heb_maping.keys()))            
            choosen_heb=arab_heb_maping[choosen_arr]
            if len(choosen_heb)==2:
              heb_res.append(choosen_heb[0])
            else:
              heb_res.append(choosen_heb)
            arab_res.append(choosen_arr)       
          heb_choosen_int=[inp_lang.char2idx[cr] for cr in heb_res]
          arab_choosen_int=[targ_lang.char2idx[cr] for cr in arab_res]              
          inp.append(heb_choosen_int)          
          target.append(arab_choosen_int)    

        inp=tf.convert_to_tensor(inp)
        target=tf.convert_to_tensor(target)
        
        with tf.GradientTape() as tape:
            predictions = model(inp)   
            cost = tf.compat.v1.losses.sparse_softmax_cross_entropy(target, predictions)
            total_loss+=cost

        grads = tape.gradient(cost, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    template = 'Epoch {} Loss {:.4f}'
    #test__CTC_letters()
    print_log_screen(template.format(epoch+1,  total_loss))


In [0]:
CELL_NAME="TRAIN SINGLE LETTERS AND TEST LETTERS"
# model=rebuild()
# test__CTC_letters()
# optimizer = tf.compat.v1.train.RMSPropOptimizer(0.001)
# pretrain_letters(10,BATCH_SIZE)
# test__CTC_letters()
# save_checkpoint("testing1")
# model=rebuild()
# test__CTC_letters()
# model=load_checkpoint()
# test__CTC_letters()

##train loop

In [0]:
CELL_NAME="DEF main TRAIN_LOOP"


#A SINGLE EPOCH
def train_loop(cur_dataset=dataset_double_kuzari,stop_loop=10000000000):
  # Training step  
  global GLOBAL_epoch
  # initializing the hidden state at the start of every epoch
  # initially hidden is None
  if STATEFUL:
    hidden = model.reset_states()
  total_loss=0
  for (batch_n, (inp, target,input_lens,target_lens)) in enumerate(cur_dataset):
        if batch_n>stop_loop:
          break
        with tf.GradientTape() as tape:
            # feeding the hidden state back into the model
            # This is the interesting step
            predictions = model(inp)                
            #labels=tf.cast(target, tf.int32) #need?  
            logits=tf.transpose(predictions,perm=[1,0,2])  
            loss=tf.nn.ctc_loss(target,logits, target_lens,input_lens,blank_index=targ_lang.char2idx[BLANK])              
            
            cost = tf.reduce_mean(loss)
            total_loss+=cost

        grads = tape.gradient(cost, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if batch_n % 10 == 0:
            template = 'Epoch {} Batch {} Loss {:.4f}'
            print_log_screen(template.format(GLOBAL_epoch+1, batch_n, cost))
  GLOBAL_epoch+=1
  return total_loss.numpy()
  


#MAIN

In [338]:
CELL_NAME="MAIN"



BEST_ACCURACY=1
f= open("my_log.txt","w+")
np.random.seed(1)
#tf.set_random_seed(1)
#tf.random.set_seed(1)
tf.compat.v1.set_random_seed(1)

mail_subject="NOT STATEFULL: pretrain letters. synt DROPOUT 0.9 no KUZARI"
mail_subject=this_time+":"+mail_subject

pretrain_letter=15
synth=True
keep_percent=0.85

description="\n"+"pretrain: "+str(pretrain_letter)+"\n"+ \
    ("no synth" if not synth else "with synth data")+ \
    "\n"+"dropout:"+str(keep_percent) +"\n"
print_log_screen(description)

#LOG Stats
losses=[]
test_losses=[]
accuracys=[]


model=rebuild()
optimizer = tf.compat.v1.train.RMSPropOptimizer(0.001)
if pretrain_letter>0:
  #optimizer = tf.train.AdamOptimizer() #adam is a bit better for this #didn't work
  print_log_screen("PRETRAIN")
  pretrain_letters(pretrain_letter,BATCH_SIZE)
  print_log_screen('-'*200)

print_log_screen("START TRAIN")


for jjj in range(10): #after each of this iterations - send mail and calc full test
  for i in range(3): #iter without sendmail and only partial test
    log_flush()
    if synth:
      dataset_double_synt=gen_dropout_all(sentences,sentences1,sentences2,sentences3,keep=keep_percent)
      loss=train_loop(dataset_double_synt,stop_loop=350)
    else:
      loss=train_loop(dataset_double_kuzari)
    #total_test_loss,total_accuracy=test_loss(single_words_test_dataset,limit=5)
    
    total_test_loss,total_accuracy=test_loss(limit=5)
    test_loss(this_dataset=test_dataset_double_rasag,limit=5)

    #losses.append(loss)
    #test_losses.append(total_test_loss)
    #accuracys.append(total_accuracy)


    # print ('Epoch {} Loss {:.4f} Test Loss {:.4f} accuracy {:.4f}' \
    #        .format(GLOBAL_epoch, loss, total_test_loss,total_accuracy))    
       
    print_log_screen('-'*200)
    test_kfir(kfir_kuzari_test,False)
    #test_kfir(kfir_kuzari_test)
    test_kfir(kfir_rasag_test,False)
    #test_kfir(kfir_rasag_test)
    print_log_screen('-'*200)
  print_log('FULL STATISTICS')
  print_log('='*200)
  test_kfir(kfir_kuzari_test,False,True)
  #test_kfir(kfir_kuzari_test)
  test_kfir(kfir_rasag_test,False,True)
  #test_kfir(kfir_rasag_test)
  #TESTING ALL EVERY OUTER LOOP 
  all_test_loss,all_accuracy=test_loss()
  #shuffle_loss,shuffle_accuracy=test_shuffle()
  all_test_loss1,all_accuracy1=test_loss(test_dataset_double_rasag)  #HAEMUNOT VEHADEOT
  #shuffle_loss1,shuffle_accuracy1=test_shuffle(shuffle_test_dataset_double_rasag)
  
  
  guide_result=test_guide()
  
  #TODO SAVE CHECKPOINT
  if all_accuracy<BEST_ACCURACY:
    save_checkpoint("improvement in accuracy. Current LER on KUZARI test data: "+str(all_accuracy))
    BEST_ACCURACY=all_accuracy

#  my_plot_save(losses,"train.png",decor='r--')
#  my_plot_save(test_losses,"test.png",decor='b-')
#  my_plot_save(accuracys,"accuracys.png",decor='g-')
  
  print_log("full test: loss ",all_test_loss," accuracy ",all_accuracy)
  print_log("full test (HAEMUNOT): loss ",all_test_loss1," accuracy ",all_accuracy1)

 # print_log("shuffle test (HAEMUNOT): loss ",shuffle_loss1," accuracy ",shuffle_accuracy1)
  print('='*200)
  print('CONTINUE TRAINING')

  # for l,t,a in zip(losses,test_losses,accuracys):
  #   print(l,t,a)
  #   f.write("%.3f %.3f %.6f\r\n" % (l,t,a))
  f.write("EPOCH "+str(GLOBAL_epoch)+'\n')
  f.write("full test: loss %.6f accuracy %.6f\r\n" % (all_test_loss,all_accuracy))
  f.write("full test (HAEMUNOT): loss  %.6f accuracy %.6f\r\n" % (all_test_loss1,all_accuracy1))
  #f.write("shuffle test (HAEMUNOT): loss %.6f accuracy %.6f\r\n" % (shuffle_loss1,shuffle_accuracy1))
  




  #[(l,t,a) for l,t,a in zip(losses,test_losses,accuracys)]
  f.flush()
  send_results(mail_subject,str(all_accuracy)+'\n'+str(all_accuracy1)+description+'\n\n'+guide_result)
f.close()
close_log()


pretrain: 15
with synth data
dropout:0.85

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (128, None, 8)            384       
_________________________________________________________________
bidirectional_44 (Bidirectio (128, None, 2048)         6352896   
_________________________________________________________________
bidirectional_45 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
bidirectional_46 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
bidirectional_47 (Bidirectio (128, None, 2048)         18886656  
_________________________________________________________________
dense_11 (Dense)             (128, None, 65)           133185    
Total params: 63,146,433
Trainable params: 63,146,433
Non-trainable params:

SMTPSenderRefused: ignored

#MAIN OUTPUT (ABOVE)

In [0]:
# test_loss()
#test_loss(test_dataset_double_rasag)

In [0]:
# test_guide()
# test_shuffle()

In [0]:
# for i in range(50):
#   train_loop()
#   test_loss(single_words_test_dataset,limit=5)
#   test_loss(limit=5)

In [0]:
  # test_loss(only_first=False)

In [0]:
test_loss(this_dataset=test_dataset_double_kuzari,only_first=False)

In [0]:
shuffle_loss,shuffle_accuracy=test_shuffle()

In [0]:
  shuffle_loss1,shuffle_accuracy1=test_shuffle(shuffle_test_dataset_double_rasag)

In [0]:
test_loss(this_dataset=test_dataset_double_rasag,only_first=False)

test_guide(limit=3)TESTTtttt#TODO


1.   varied length for data - to makes the system more robust for sentneces with different lengths. can do this with SENTENCE_LIMIT=20 set to random limit when sentences length exceedes current limit

2.   abstraction for the testing functions (see comparesment in notpad++)

3.   try TPU

4.   new idea: input - arab baseline. train network to correct it

5.    predict only middle word. input (1 true arab words) - (2 arab baseline word) - (3 true arab words) output - the middle word in corrected arab.

or calc results only on middle word(s)

6.   transformer (see tf tutorial)

7.    NEW AND INTERESTING!!!!!: add space to each line at start and at end
so the network knows this is end of word!




#OLD STAFF

In [0]:
# CELL_NAME="SMALL TRY"

# this_string='وأهل الأديان ثمّ على'
# this_string='سُئِلْتُ عمّا عنديَ من الاحتجاج'
# this_string='ثمّ'

# #this_string='كان عند مَلِك الخَزَرِ الداخل'
# print_log_screen(len(this_string))
# this_string=normalize_unicode(remove_arab_nikud(this_string)) #new!!!!

# print_log_screen(len(this_string))
# this_string

In [0]:
# CELL_NAME="TRY SYNTH"

# #ACTIVATE
# ibnsina_text=load_text_for_synth()

# #STATSTICS OF SYNTH TEXT
# sina_vocab=sorted(set(ibnsina_text))

# print_log("NOT IN LETTER LIST:")
# for c in sina_vocab:
#    if c not in targ_lang.char2idx:
#       print_log("(",c,")")

# print_log("\nLETTER COUNTS")
# for i in range(len(sina_vocab)):
#   print_log(LTRchar,i,'"',sina_vocab[i],'"',ibnsina_text.count(sina_vocab[i])) #64 is shadda   

# #ibnsina_text=ibnsina_text.replace("\r\n\r\n","\r\n").replace("\r\n"," \r\n ") #TODO rethink this
# #sina_words=ibnsina_text.split(" ")
# #ibnsina_text=ibnsina_text.replace("\r\n\r\n","\r\n").replace("\r\n",". ") #TODO rethink this
# sina_words=ibnsina_text.split() #for removing also newlines
# print_log(ibnsina_text[:100])
# sina_words[0:10]

##Shuffled test (NOT USED NOW)

In [0]:
# CELL_NAME="SHUFFLED TEST"

# def get_shuffled_word_pairs(test_dataset):
  
#   val_inputs_list=[]
#   val_outputs_list=[]

#   for i,j,l1,l2 in test_dataset:
#     for tt in range(BATCH_SIZE):
#      # print_log(i[tt],j[tt])
#       i_prediction=print_by_idx_CTC(tf.constant(i[tt]),inp_lang.idx2char,l1[tt])
#       j_prediction=print_by_idx_CTC(tf.constant(j[tt]),targ_lang.idx2char,l2[tt])      
#       if (len(i_prediction.split())==len(j_prediction.split())):
#         val_inputs_list+=i_prediction.split()
#         val_outputs_list+=j_prediction.split()
    
#   print_log("len(val_inputs_list),len(val_outputs_list)",len(val_inputs_list),len(val_outputs_list)) #10836 10836

#   word_pairs=list(zip(val_inputs_list,val_outputs_list))
#   print_log("BEFORE SHUFFLE")
#   for i in word_pairs[:5]:
#     print_log(i)
#   random.shuffle(word_pairs)
#   print_log("AFTER SHUFFLE")
#   for i in word_pairs[:5]:
#     print_log(i)
#   return word_pairs

# #TESTING
# #word_pairs1=get_shuffled_word_pairs(test_dataset_double_rasag.take(1))


In [0]:
# CELL_NAME="DEF GET SHUFFLE DATA"
# def get_shuffled_data(word_pairs):

#   accum=0
#   heb_acum=""
#   arab_acum=""
#   results_line=[]
#   for i,j in word_pairs:
#     if accum>19:
#       results_line.append(undouble_hebrew(heb_acum)+'\t'+arab_acum)
#       assert(len(i)%2==0)
#       accum=len(i)/2
#       heb_acum=i
#       arab_acum=j
#     else:
#       heb_acum+=" "+i
#       arab_acum+=" "+j 
#       assert(len(i)%2==0)
#       accum += len(i)/2 + 1;
#   results_line.append(heb_acum+'\t'+arab_acum)  #needed?

#   print_log("len(results_line)",len(results_line)) # 2175 before was: 2185 lines 


#   input_tensor_shuffle, target_tensor_shuffle \
#   ,input_lenghts_shuffle,target_lengths_shuffle = load_dataset(create_dataset(results_line))

#   print_log("len(input_tensor_shuffle), len(input_lenghts_shuffle)",len(input_tensor_shuffle), len(input_lenghts_shuffle))
#   print_log("len(target_tensor_shuffle),  len(target_lengths_shuffle)",len(target_tensor_shuffle), len(target_lengths_shuffle))

#   BUFFER_SIZE = len(target_lengths_shuffle)

#   shuffle_test_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_shuffle,
#                                                             target_tensor_shuffle,
#                                                             input_lenghts_shuffle,
#                                                             target_lengths_shuffle)).shuffle(BUFFER_SIZE)

#   shuffle_test_dataset_double=shuffle_test_dataset.batch(BATCH_SIZE, drop_remainder=True)
#   return shuffle_test_dataset_double



In [0]:
CELL_NAME="GEN SHUFFLED DATA"

# word_pairs=get_shuffled_word_pairs(test_dataset_double_kuzari)
# shuffle_test_dataset_double=get_shuffled_data(word_pairs)
# view_data(shuffle_test_dataset_double)


# word_pairs1=get_shuffled_word_pairs(test_dataset_double_rasag)
# shuffle_test_dataset_double_rasag=get_shuffled_data(word_pairs1)
# view_data(shuffle_test_dataset_double_rasag)




##test shuffle

In [0]:
# CELL_NAME="DEF TEST_SHUFFLE"
# def test_shuffle(data=shuffle_test_dataset_double,limit=False):
#   return test_loss(this_dataset=data,limit=limit)
# #shuffle_loss,shuffle_accuracy=test_shuffle(limit=3)


##Single words test (NOT USED NOW)

In [0]:
# CELL_NAME="GEN SINGEL WORDS"

# # results_line=[]
# # for i,j in word_pairs:
# #     results_line.append(undouble_hebrew(i)+'\t'+j)

# # print_log("len(results_line)",len(results_line)) 


# # input_tensor_shuffle, target_tensor_shuffle \
# # , input_lenghts_shuffle,target_lengths_shuffle = load_dataset(create_dataset(results_line))

# # print_log("len(input_tensor_shuffle), len(target_tensor_shuffle), len(input_lenghts_shuffle), len(target_lengths_shuffle)",len(input_tensor_shuffle), len(target_tensor_shuffle), len(input_lenghts_shuffle), len(target_lengths_shuffle))

# # BUFFER_SIZE = len(target_lengths_shuffle)

# # single_words_test_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_shuffle,
# #                                                                 target_tensor_shuffle,
# #                                                                 input_lenghts_shuffle,
# #                                                                 target_lengths_shuffle)).shuffle(BUFFER_SIZE)

# # single_words_test_dataset=single_words_test_dataset.batch(BATCH_SIZE, drop_remainder=True)

# # view_data(single_words_test_dataset)